# Import Things

In [28]:
import numpy as np
from random import randint
import os
import json
import settings
import pickle
import nltk.data
from pyvi import ViTokenizer

## Read file

In [29]:
class FileReader(object):
    def __init__(self, filePath, encoder = None):
        self.filePath = filePath
        self.encoder = encoder if encoder != None else 'utf-8'
    
    def read(self):
        with open(self.filePath, 'r') as f:
            s = f.read()
        return s
    
    def content(self):
        s = self.read()
        return s.decode(self.encoder)
    
    def read_json(self):
        s = ''
        with open(self.filePath, 'r', encoding=self.encoder) as f:
            s = json.load(f)
        return s

    def read_stopwords(self):
        with open(self.filePath, 'r') as f:
            stopwords = set([w.strip().replace(' ', '_') for w in f.readlines()])
        return stopwords

    def load_dictionary(self):
        return corpora.Dictionary.load_from_text(self.filePath)



## File Store

In [30]:
class FileStore(object):
    def __init__(self, filePath, data = None):
        self.filePath = filePath
        self.data = data

    def store_json(self):
        with open(self.filePath, 'w') as outfile:
            json.dump(self.data, outfile)

    def store_dictionary(self, dict_words):
        dictionary = corpora.Dictionary(dict_words)
        dictionary.filter_extremes(no_below=20, no_above=0.3)
        dictionary.save_as_text(self.filePath)
    
    def save_pickle(self, obj):
        outfile = open(self.filePath, 'wb')
        fastPickler = pickle.Pickler(outfile, pickle.HIGHEST_PROTOCOL)
        fastPickler.fast = 1
        fastPickler.dump(obj)
        outfile.close()

## NLP


In [31]:
from pyvi import ViTokenizer, ViPosTagger
import numpy as np
import string
import re


class NLP(object):
    def __init__(self, text=None):
        self.text = text
        self.stopwords = FileReader(settings.STOP_WORDS).read_stopwords()
        # print(str(len(self.stopwords)))
        self.newWords = []
        
    def remove_tags(self, raw_html):
        cleanr = re.compile('<.*?>')
        cleantext = re.sub(cleanr, '', raw_html )
        return cleantext

    def remove_links(self, text):
        text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
        return text

    def remove_punctuation(self, text):
        return text.translate(str.maketrans('', '', string.punctuation))
        

    def remove_stopwords(self, words, stopwords):
        important_words = []
        for index in range(len(words)):
            if (words[index] not in stopwords ):
                important_words.append(words[index])
        self.tokens = important_words
        return important_words
    
    def lower_sentences(self, text):
         return text.lower()

    def tokenize_sentences(self, text):
        return ViTokenizer.tokenize(text)

    def segment_word(self, tokens):
        return ViPosTagger.postagging(tokens)
    
    def keep_verb_noun(self, postags):
        res = []
        for index in range(len(postags[0])):
            if postags[1][index] == 'N' or postags[1][index] == 'V' or postags[1][index] == 'Np':
                res.append(postags[0][index])
        return res

    def remove_digits(self, tokens):
        new_words = []
        for index in range(len(tokens)):
            if not tokens[index].isdigit():
                new_words.append(tokens[index])
        return new_words
    
    def count_tokens(self, tokens):
        unique, counts = np.unique(tokens, return_counts=True)
        return dict(zip(unique, counts))
    
    def retrieve_tokens_times(self, tokens, n):
        tokens = {k: v for k, v in self.count_tokens(tokens).items() if int(v) >= n}
        return tokens 
        
    def preprocessText(self, text):
        processed_text = text
        processed_text = self.remove_tags(processed_text)
        processed_text = self.lower_sentences(processed_text)
        processed_text = self.remove_links(processed_text)
        processed_text = self.remove_punctuation(processed_text)
        tokens = self.tokenize_sentences(processed_text)
        tokens = self.segment_word(tokens)
        # print(tokens)
        tokens = self.keep_verb_noun(tokens)
        important_tokens = self.remove_stopwords(tokens, self.stopwords)
        important_tokens = self.remove_digits(important_tokens)
        important_tokens = self.retrieve_tokens_times(important_tokens, 4)
        self.tokens = important_tokens
        return self.tokens

    def __repr__(self):
        return self.text

## DATASET PREPARE

In [12]:
url = "https://dantri.com.vn/giai-tri/bieu-tuong-goi-cam-emily-ratajkowski-sinh-con-dau-long-20210313082417860.htm"
print(url.split('/'))

['https:', '', 'dantri.com.vn', 'giai-tri', 'bieu-tuong-goi-cam-emily-ratajkowski-sinh-con-dau-long-20210313082417860.htm']


In [32]:
import time
start_time = time.time()
json = FileReader(settings.DIR_PATH + '//'+  'data.json').read_json()
data = list()
categories = []
cate = set()
categories_data = []
for article in json:
    nlp = NLP(text=article['content'])
    tokens = nlp.preprocessText(article['content'])
    # key_in_article = []
    # for key, value in tokens.items():
        # key_in_article.append(key)
    topic = article['url'].split('/')
    categories_data.append(topic[3])
    cate.add(topic[3])  
    key_in_article = ' '.join(tokens) 
    data.append(key_in_article)
print('duration: {}'.format(time.time() - start_time))
categories = list(cate)
        


duration: 354.5249581336975


In [33]:
print(categories)
print(len(categories))

['du-lich', 'tinh-yeu-gioi-tinh', 'suc-khoe', 'kinh-doanh', 'du-hoc', 'phap-luat', 'an-sinh', 'bat-dong-san', 'xa-hoi', 'ban-doc', 'the-thao', 'khoa-hoc-cong-nghe', 'van-hoa', 'lao-dong-viec-lam', 'nhip-song-tre', 'the-gioi', 'doi-song', 'blog', 'tam-long-nhan-ai', 'giai-tri', 'suc-manh-so', 'giao-duc-huong-nghiep', 'o-to-xe-may', 'tuyen-sinh', 'multimedia']
25


In [107]:
print(data)
print(len(categories_data))

ả hồng kháng_nghị kiểm_sát lọt minh mừng ngô phê_duyệt quản_lý sai_phạm sơ_thẩm thiết_kế thượng thẩm_định trung_tâm trách_nhiệm tỉnh tội_phạm ubnd viện văn xây_dựng đầu_tư đồng', 'băng_nhóm chuyên_án công_an cơ_quan cảnh_sát cẩm dũng hành_vi phòng quảng_ninh quốc trần tỉnh', 'bị_can chi_nhánh công_an cơ_quan cảnh_sát hành_vi khởi_tố nguyễn nhà_nước phan_thiết quyết_định sai_phạm thị tiền tp trách_nhiệm tỉnh văn_phòng xác_định điều_tra đăng_ký đất_đai', 'bắt campuchia cách_ly công_an loan lâm nam nhập_cảnh việt xã điều_tra', 'công_an huyện huệ minh thượng tý vé_số văn xã', 'csgt mân nam thanh_niên xe_máy đi đường đại_úy', 'an_ninh chính_phủ công cơ_quan hoạt_động khởi_tố lâm_thời nam ngô phú quốc_gia tham_gia trứ tỉnh tổ_chức việt xã_hội yên điều_tra', 'bình cao_su công_an dao phước sảnh tỉnh xã án đi điều_tra điểu đối_tượng', 'cường dương giam huyện phòng thái văn xã đỗ', 'bms bệnh_viện bị_can công_an công_ty cơ_quan cảnh_sát khởi_tố lệnh mai quyết_định thị tài_chính vfs điều_tra', 'bị

### Save dataset with pickle

In [34]:
fileStore = FileStore(settings.DIR_PATH + "//"+ 'xnew.pkl')
fileStore.save_pickle(data)
fileStore = FileStore(settings.DIR_PATH + "//"+ 'ynew.pkl')
fileStore.save_pickle(categories_data)